This script uses pandas to backfill (tugboat-related) metadata about all surveys that exist on NCEI.
This metadata is expected to be extracted from NCEI into an Excel file using the query below:

`
select * from cruise.wcsd_all_survey_summary_msql
where source_group like '%NMFS%'
order by source_name, dataset_name
`

The excel file is then parsed through in this script, and the resultant, processed dataframe is then uploaded to BigQuery.
The Tugboat metadata format is outlined below:

Fields:
* Cruise ID - string
* Segment ID - string
* Master (default) release date - date, when these data should be released
* Ship name - string (controlled vocabulary)
* Departure port - string
* Arrival port - string
* Departure date - date
* Arrival date - date
* Sea area - string (controlled vocabulary)
* Cruise title - string
* Cruise purpose - string
* Cruise description - string
* Sponsors - list of organization names (strings) (controlled vocabulary)
* Funders - list of organization names (strings) (controlled vocabulary)
* Scientists - list of person objects (controlled vocabulary)
* Projects - list of project names (strings)
* Metadata author - person object (controlled vocabulary)
* Instruments - list of instrument objects (controlled vocabulary)
* Documents URI - URI to documents files (bucket)

Instrument Object:
* instrument - instrument name (string) (controlled vocabulary)
* release date - date, when these data should be released (overrides master release date if specified)
* status - enum representing processing type (Raw, Processed, or Products)
* calibration state - enum representing calibration performed on instrument (Calibrated w/ calibration data, Calibrated w/o calibration data, Uncalibrated, Uncalibrated w/ calibration data, * Unknown)
* calibration date - date, when instrument calibration was performed
* calibration reports URI - URI to calibration reports (bucket)
* calibration data / support URI - URI to calibration data / support files (bucket)
* Data details - string
* Data URI - URI to instrument data (bucket)
* Ancillary data details - string
* Ancillary data URI - URI to ancillary data (bucket)

Person Object:
* name - person name (string)
* organization - organization person is associated with (string)

In [7]:
import pandas as pd

In [8]:
file_path = r"C:\Users\hannan.khan\Downloads\WCSD_DB_ALL_SUMMARY.xlsx"
df = pd.read_excel(file_path)
df.head()

c:\Users\hannan.khan\AppData\Local\conda\conda\envs\aalibrary\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,OBJECTID,WCS_ID,DATASET_NAME,INSTRUMENT_NAME,PROJECT_NAME,SCIENTIST_NAME,SOURCE_NAME,SOURCE_GROUP,CRUISE_NAME,PLATFORM_NAME,...,MIN_PULSE_LENGTH,MAX_PULSE_LENGTH,ANCILLARY,SHAPE,GEOM_TYPE,CLOUD_PATH,FILE_COUNT,DATASET_SIZE,INGEST_TIME,ARCHIVE_DATE
0,10047,10047,AI04GL_ES60,ES60,|Bottom trawl survey of groundfish resources i...,|Groundfish Assessment Program|,|AFSC|,|NMFS|,AI04GL,Gladiator,...,NaN,NaN,NaN,NaN,line,https://noaa-wcsd-pds.s3.amazonaws.com/index.h...,50,23819849116,NaN,2022-11-30 16:59:22
1,9902,9902,AI04SS_ES60,ES60,|Bottom trawl survey of groundfish resources i...,|Groundfish Assessment Program|,|AFSC|,|NMFS|,AI04SS,Sea Storm,...,NaN,NaN,NaN,"MDSYS.SDO_GEOMETRY(2006, 8307, NULL, MDSYS.SDO...",line,https://noaa-wcsd-pds.s3.amazonaws.com/index.h...,118,67346349156,NaN,2022-09-13 18:01:15
2,9903,9903,AI06GL_ES60,ES60,|Bottom trawl survey of groundfish resources i...,|Groundfish Assessment Program|,|AFSC|,|NMFS|,AI06GL,Sea Storm,...,NaN,NaN,NaN,NaN,NaN,https://noaa-wcsd-pds.s3.amazonaws.com/index.h...,324,16165358520,NaN,2022-09-13 18:02:57
3,9904,9904,AI06SS_ES60,ES60,|Bottom trawl survey of groundfish resources i...,|Groundfish Assessment Program|,|AFSC|,|NMFS|,AI06SS,Sea Storm,...,NaN,NaN,NaN,NaN,NaN,https://noaa-wcsd-pds.s3.amazonaws.com/index.h...,1431,74291968696,NaN,2022-09-13 18:20:32
4,9905,9905,AI10OE_ES60,ES60,|Bottom trawl survey of groundfish resources i...,|Groundfish Assessment Program|,|AFSC|,|NMFS|,AI10OE,Ocean Explorer,...,NaN,NaN,NaN,NaN,line,https://noaa-wcsd-pds.s3.amazonaws.com/index.h...,1082,90764737141,NaN,2022-09-13 19:30:55


In [9]:
sorted(list(df.columns))

['ANCILLARY',
 'ARCHIVE_DATE',
 'ARRIVAL_PORT',
 'CAL_STATE_NAME',
 'CAL_STATE_VALUE',
 'CITATION_LINK',
 'CITATION_TEXT',
 'CLOUD_PATH',
 'CRUISE_NAME',
 'DATASET_NAME',
 'DATASET_SIZE',
 'DEPARTURE_PORT',
 'DISPLAY_FREQUENCY',
 'END_DATE',
 'FILE_COUNT',
 'FREQUENCY',
 'GEOM_TYPE',
 'INGEST_TIME',
 'INSTRUMENT_NAME',
 'MAX_FREQ',
 'MAX_POWER',
 'MAX_PULSE_LENGTH',
 'MIN_FREQ',
 'MIN_POWER',
 'MIN_PULSE_LENGTH',
 'OBJECTID',
 'PLATFORM_NAME',
 'PROJECT_NAME',
 'PUBLISH',
 'PUBLISH_DATE',
 'SCIENTIST_NAME',
 'SHAPE',
 'SOURCE_GROUP',
 'SOURCE_NAME',
 'START_DATE',
 'WCS_ID']

In [10]:
# Convert all columns that have string values to a str type.
columns_w_str_values = set()
# Find which columns have string values.
for col in df.columns:
    values_list = df[col].tolist()
    for value in values_list:
        if isinstance(value, str):
            columns_w_str_values.add(col)
# Convert these columns to str type.
for col in columns_w_str_values:
    print(f"Converting column {col} to str type.")
    df[col] = df[col].astype(str)

Converting column CAL_STATE_NAME to str type.
Converting column ANCILLARY to str type.
Converting column CLOUD_PATH to str type.
Converting column CITATION_TEXT to str type.
Converting column CITATION_LINK to str type.
Converting column INSTRUMENT_NAME to str type.
Converting column SHAPE to str type.
Converting column DEPARTURE_PORT to str type.
Converting column DISPLAY_FREQUENCY to str type.
Converting column SOURCE_NAME to str type.
Converting column SCIENTIST_NAME to str type.
Converting column DATASET_NAME to str type.
Converting column FREQUENCY to str type.
Converting column PROJECT_NAME to str type.
Converting column ARRIVAL_PORT to str type.
Converting column CRUISE_NAME to str type.
Converting column PLATFORM_NAME to str type.
Converting column PUBLISH to str type.
Converting column SOURCE_GROUP to str type.
Converting column GEOM_TYPE to str type.


In [11]:
# Finding out which columns have multiple values based on the '|' delimiter.
columns_w_multiple_values_in_row = set()
for col in df.columns:
    values_list = df[col].tolist()
    for value in values_list:
        if type(value) == str:
            if '|' in value:
                columns_w_multiple_values_in_row.add(str(col))
print(f"COLUMNS WITH MULTIPLE VALUES")
print(columns_w_multiple_values_in_row)

COLUMNS WITH MULTIPLE VALUES
{'SCIENTIST_NAME', 'FREQUENCY', 'PROJECT_NAME', 'INSTRUMENT_NAME', 'SOURCE_GROUP', 'SOURCE_NAME'}


In [12]:
def parse_multiple_values(s: str = "") -> str:
    """Used for parsing through multiple values in a string using the '|' delimiter.
    NOTE: Some strings will begin and end with the delimiter, such as `|NEFSC|`.
    
    Returns:
        List[str]: A list of values parsed from the string.
    """

    s = s.strip('|')  # Remove leading and trailing delimiters
    l = [value.strip() for value in s.split('|') if value.strip()]  # Remove empty strings
    return ','.join(l)  # Join the values with a comma

In [13]:
# Applying a function to parse multiple values in a column over all columns with multiple values.
for col in df.columns:
    if col in columns_w_multiple_values_in_row:
        print(col)
        df[col] = df[col].apply(parse_multiple_values)
df.iloc[432]["INSTRUMENT_NAME"]

INSTRUMENT_NAME
PROJECT_NAME
SCIENTIST_NAME
SOURCE_NAME
SOURCE_GROUP
FREQUENCY


'EK500,EK60'

In [19]:
# COLUMNS WITH MULTIPLE VALUES
# {'SOURCE_NAME', 'PROJECT_NAME', 'INSTRUMENT_NAME', 'SOURCE_GROUP', 'FREQUENCY', 'SCIENTIST_NAME'}

# Create DF to add every row of metadata to.
full_df = pd.DataFrame(
    columns=[
        "CRUISE_ID",
        "SEGMENT_ID",
        "PACKAGE_ID",
        "MASTER_RELEASE_DATE",
        "SHIP",
        "SHIP_UUID",
        "DEPARTURE_PORT",
        "DEPARTURE_DATE",
        "ARRIVAL_PORT",
        "ARRIVAL_DATE",
        "SEA_AREA",
        "CRUISE_TITLE",
        "CRUISE_PURPOSE",
        "CRUISE_DESCRIPTION",
        "METADATA_AUTHOR",
        "SPONSORS",
        "FUNDERS",
        "SCIENTISTS",
        "PROJECTS",
        "INSTRUMENTS",
        "PACKAGE_INSTRUMENTS",
        "CALIBRATION_FILE_PATH",
    ]
)

i = 0
for idx, row in df.iterrows():
    cruise_id = row["CRUISE_NAME"]
    # the echosounder used
    segment_id = row["INSTRUMENT_NAME"]
    # This is the cruise_id and the segment_id concatenated with an underscore. Used as a prefix for file names in NCEI.
    package_id = row["DATASET_NAME"]
    # TODO: verify that we can use "PUBLISH_DATE" as the master_release_date
    master_release_date = row["PUBLISH_DATE"]
    ship = row["PLATFORM_NAME"]
    # TODO: not available
    ship_uuid = ""
    departure_port = row["DEPARTURE_PORT"]
    # TODO: verify that we can use "START_DATE" as the departure_date
    departure_date = row["START_DATE"]
    arrival_port = row["ARRIVAL_PORT"]
    # TODO: verify that we can use "END_DATE" as the arrival_date
    arrival_date = row["END_DATE"]
    # TODO: not available
    sea_area = ""
    # TODO: has multiple names sometimes that need to be parsed.
    cruise_title = row["PROJECT_NAME"]
    # TODO: not available
    cruise_purpose = ""
    # TODO: not available
    cruise_description = ""
    # TODO: not available
    metadata_author = ""

    # TODO: these have multiple names sometimes that need to be parsed.
    sponsors = row["SOURCE_NAME"]
    funders = row["SOURCE_NAME"]
    scientists = row["SCIENTIST_NAME"]
    projects = row["PROJECT_NAME"]
    instruments = row["INSTRUMENT_NAME"]
    package_instruments = row["INSTRUMENT_NAME"]
    # TODO: validate that we can just use the s3 cloud path for the calibration file paths.
    calibration_file_path = row["CLOUD_PATH"]

    ncei_survey_level_metadata_json = {
        "CRUISE_ID": cruise_id,
        "SEGMENT_ID": segment_id,
        "PACKAGE_ID": package_id,
        "MASTER_RELEASE_DATE": master_release_date,
        "SHIP": ship,
        "SHIP_UUID": ship_uuid,
        "DEPARTURE_PORT": departure_port,
        "DEPARTURE_DATE": departure_date,
        "ARRIVAL_PORT": arrival_port,
        "ARRIVAL_DATE": arrival_date,
        "SEA_AREA": sea_area,
        "CRUISE_TITLE": cruise_title,
        "CRUISE_PURPOSE": cruise_purpose,
        "CRUISE_DESCRIPTION": cruise_description,
        "METADATA_AUTHOR": metadata_author,
        "SPONSORS": sponsors,
        "FUNDERS": funders,
        "SCIENTISTS": scientists,
        "PROJECTS": projects,
        "INSTRUMENTS": instruments,
        "PACKAGE_INSTRUMENTS": package_instruments,
        "CALIBRATION_FILE_PATH": calibration_file_path,
    }

    ncei_survey_level_metadata_df = pd.json_normalize(
        ncei_survey_level_metadata_json
    )

    full_df = pd.concat([full_df, ncei_survey_level_metadata_df], ignore_index=True)

    i+=1
    if i > 10:
        break

full_df.head()

C:\Users\hannan.khan\AppData\Local\Temp\ipykernel_2684\3011880258.py:100: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_df = pd.concat([full_df, ncei_survey_level_metadata_df], ignore_index=True)


,CRUISE_ID,SEGMENT_ID,PACKAGE_ID,MASTER_RELEASE_DATE,SHIP,SHIP_UUID,DEPARTURE_PORT,DEPARTURE_DATE,ARRIVAL_PORT,ARRIVAL_DATE,...,CRUISE_PURPOSE,CRUISE_DESCRIPTION,METADATA_AUTHOR,SPONSORS,FUNDERS,SCIENTISTS,PROJECTS,INSTRUMENTS,PACKAGE_INSTRUMENTS,CALIBRATION_FILE_PATH
0,AI04GL,ES60,AI04GL_ES60,2022-06-23,Gladiator,,"Dutch Harbor, US AK",2004-06-02,"Dutch Harbor, US AK",2004-08-02,...,,,,AFSC,AFSC,Groundfish Assessment Program,Bottom trawl survey of groundfish resources in...,ES60,ES60,https://noaa-wcsd-pds.s3.amazonaws.com/index.h...
1,AI04SS,ES60,AI04SS_ES60,2022-06-13,Sea Storm,,"Dutch Harbor, US AK",2004-06-02,"Dutch Harbor, US AK",2004-08-01,...,,,,AFSC,AFSC,Groundfish Assessment Program,Bottom trawl survey of groundfish resources in...,ES60,ES60,https://noaa-wcsd-pds.s3.amazonaws.com/index.h...
2,AI06GL,ES60,AI06GL_ES60,2022-06-21,Sea Storm,,"Dutch Harbor, US AK",2006-06-01,"Dutch Harbor, US AK",2006-07-06,...,,,,AFSC,AFSC,Groundfish Assessment Program,Bottom trawl survey of groundfish resources in...,ES60,ES60,https://noaa-wcsd-pds.s3.amazonaws.com/index.h...
3,AI06SS,ES60,AI06SS_ES60,2022-06-21,Sea Storm,,"Dutch Harbor, US AK",2006-06-01,"Dutch Harbor, US AK",2006-08-09,...,,,,AFSC,AFSC,Groundfish Assessment Program,Bottom trawl survey of groundfish resources in...,ES60,ES60,https://noaa-wcsd-pds.s3.amazonaws.com/index.h...
4,AI10OE,ES60,AI10OE_ES60,2022-06-21,Ocean Explorer,,"Dutch Harbor, US AK",2010-06-06,"Dutch Harbor, US AK",2010-08-12,...,,,,AFSC,AFSC,Groundfish Assessment Program,Bottom trawl survey of groundfish resources in...,ES60,ES60,https://noaa-wcsd-pds.s3.amazonaws.com/index.h...


In [ ]:
parse_multiple_values(df.iloc[432]["INSTRUMENT_NAME"])